In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import pathlib
import textwrap
import os
import csv
import pandas as pd
import time
import google.generativeai as genai

GOOGLE_API_KEY= '<<ENTER YOUT API KEY>>'

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
df = pd.read_csv('./result_ASR.csv')
original = df['transcription']
beam_result1 = df['whisper_base_predSE'] # The 1st candidate for ASR, which contains an error
beam_result2 = df['whisper_small_predSE'] # The 2nd candidate for ASR, which contains an error

In [ ]:
from requests.exceptions import ConnectionError

gemini_result = []
max_retries = 3

for i in range(0, len(original)):
    instruction = "You are an AI model that corrects Automatic Speech Recognition results and converts them into LaTeX. Let's think step by step. I will give you two candidates from the Automatic Speech Recognition results. Based on these two ASR results, make the necessary corrections and then output the corresponding LaTeX code for the formula. I will show you one example, so refer to it and convert it into LaTeX. You just have to tell me the answer. Don't tell me anything else, just tell me the LaTeX code."
    prompt = f"""
    Input : a plus seven y plus ten z equals z || a plus 7y plus f z equals 0
    Output : $ a + 7y + 10z = 0 $
    Input : {beam_result1[i]} || {beam_result2[i]}
    Output : (Tell me Your answer.)
    """
    model = genai.GenerativeModel('gemini-pro')
    full_prompt = f"{instruction}\n{prompt}"
    # model call
    for attempt in range(max_retries):
        try:
            response = model.generate_content(full_prompt)
            result = response.text
            gemini_result.append(result)
            print(f"{result}")
            break  # If successful, stop iterating.
        except ConnectionError as e:
            print(f"ConnectionError occurred on attempt {attempt + 1}: {e}")
            if attempt < max_retries - 1:
                time.sleep(5)  # Retry in 5 seconds
            else:
                print(f"Failed after {max_retries} attempts.")
                gemini_result.append("<ERROR>")  # If error occurs, add <ERROR> to the result.
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            if attempt < max_retries - 1:
                time.sleep(5)  # Retry in 5 seconds
            else:
                print(f"Failed after {max_retries} attempts.")
                gemini_result.append("<ERROR>")  # If error occurs, add <ERROR> to the result.
    time.sleep(1)

In [ ]:
df["gemini_LaTeX_result"] = gemini_result
df.to_csv('gemini_LaTeX_result.csv', index=False)